In [1]:
print("hi")

[1] "hi"


In [3]:
require(ggplot2)

data_dir="/oak/stanford/groups/smontgom/erobb/data"

files = c("ESN.SVs.11.5.txt",
        "GWD.SVs.13.6.txt",
        "LWK.SVs.10.4.txt",
        "MKK.SVs.14.4.txt",
        "YRI.SVs.8.2.txt",
        "MSL.SVs.11.4.txt")
names(files) <- c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")

#pops = c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")
pop = "GWD"

# Read in deseq2 expression data
expr_file=sprintf('%s/deseq/deseq2ScaledGeneCounts.%s.bed',data_dir,pop)
expr = read.delim(expr_file)
row.names(expr) = expr[,"ID"] # set row name to gene ID

# Read in covariates matrix
covs_file = sprintf("/oak/stanford/groups/smontgom/mdegorte/durga/africa/fastqtl/SVs/%s/%s",pop,files[[pop]])
covs = read.table(covs_file, header = T, sep = '\t', row.names = 1)

# Read in top eQTL
eqtl_file=sprintf('%s/eqtls/sorted.byGene.dist.hwe.af.%s.top1.eQTL.nominal.hg38a.txt', data_dir, pop)
eqtl_df = read.table(eqtl_file, header = T, row.names = 'feature')

# Read in vcf data for each top eQTL
vcf_file = sprintf("%s/eqtls/AF.all.%s.hg38aID.eQTLs.ba.vcf", data_dir, pop)
vcf_df = read.table(vcf_file, header=T, skip=0, row.names="GENE")

# Filter to IDs with available covariates
id_keep = colnames(expr)[5:length(colnames(expr))]
id_keep = intersect(id_keep,colnames(covs))
covs = covs[,id_keep]
expr = expr[,id_keep]

# Subset to protein-coding & lincRNA genes
pc = eqtl_df["geneType"] == "protein_coding"
linc = eqtl_df["geneType"] == "lincRNA"
eqtl_df = eqtl_df[pc | linc,]

# 1 if alt allele is present, 0 otherwise
vcf_df[,id_keep] = sapply(vcf_df[,id_keep] == '0|0', as.numeric)
vcf_df = vcf_df[,id_keep]

# Scale to mean 0 variance 1 (over each gene)
expr = scale(t(expr))

# # For each gene in the expression file, perform a linear regression 
resids = matrix(, ncol = ncol(expr), nrow = nrow(expr))
rownames(resids) = rownames(expr)
colnames(resids) = colnames(expr)

for(i in 1:ncol(expr)){ # for each gene! seems slow but not too bad
    gene = colnames(expr)[i]
    if (!all(is.na(vcf_df[gene,]))) {
        gcovs = rbind(covs, vcf_df[1,])
    } else {
        gcovs = covs
    }
    data = as.data.frame(cbind(expr[, i], t(gcovs)))
    colnames(data) = c('deseq2ScaledGeneCounts', rownames(gcovs))
    model = lm(deseq2ScaledGeneCounts ~ ., data = data)
    resids[, i] = model$residuals
}
# Center and scale, then transpose
resids = t(scale(resids))

# Write out the residuals
out_file = sprintf('%s/eoutliers/%s_exprResiduals.tsv', data_dir, pop)
#write.table(matrix(c('Id', colnames(resids)), nrow = 1), out_file, quote = F, row.names = F, col.names = F, sep = '\t')
#write.table(resids, out_file, row.names = T, col.names = F, quote = F, sep = '\t', append = T)



In [18]:
#which(is.na(resids))

i = 15329
gene = colnames(expr)[i]
if (!all(is.na(vcf_df[gene,]))) {
    gcovs = rbind(covs, vcf_df[1,])
} else {
    print("hi")
}
data = as.data.frame(cbind(expr[, i], t(gcovs)))
colnames(data) = c('deseq2ScaledGeneCounts', rownames(gcovs))
model = lm(deseq2ScaledGeneCounts ~ ., data = data)
model$residuals
#resids[,15329:15330]
#expr[,15329:15330]

[1] "hi"


HG02574       HG02623       HG02675       HG02679       HG02769 
-4.860428e-02  3.438980e-01  1.665261e-01  2.174676e-01 -5.694810e-01 
      HG02819       HG02881       HG03024       HG03247       HG03025 
 1.815059e-01  4.248592e-01  5.177584e-01  1.270651e-01 -9.526105e-01 
      HG02582       HG02624       HG02702       HG02771       HG02820 
 8.766412e-01  1.656091e-01 -6.335947e-03  8.822537e-01  1.127426e-01 
      HG02882       HG03258       HG03027       HG02461       HG02583 
-9.583531e-01  2.556703e-01 -1.089156e+00  6.712721e-01  9.633425e-02 
      HG02628       HG02703       HG02772       HG02804       HG02836 
 7.528027e-02 -3.234879e-01  1.853069e-01 -1.711663e-01 -5.158100e-02 
      HG02884       HG03259       HG03028       HG02462       HG02585 
-5.284189e-17 -2.218221e-01  2.916346e-01 -1.093999e-03 -1.325414e-01 
      HG02629       HG02715       HG02798       HG02805       HG02837 
 7.764038e-01 -1.903439e+00 -9.098147e-01  3.473567e-03 -4.386142e-01 
      HG02885       HG03039       HG03538       HG02464       HG02586 
 2.080456e-01  2.937939e-02  8.071507e-01  1.554709e-01  2.526178e-01 
      HG02634       HG02716       HG02799       HG02807       HG02839 
 1.238746e-01  4.784040e-01  2.384147e-01 -2.677952e-01 -1.370312e+00 
      HG02887       HG03040       HG03539       HG02465       HG02588 
 2.347517e-01  3.987631e-02 -8.514506e-01 -4.009796e-02  4.397473e-01 
      HG02635       HG02721       HG02808       HG02840       HG02888 
-1.626259e-01  2.108621e-01 -2.412128e-01 -4.058251e-01 -6.901247e-01 
      HG03045       HG02561       HG02589       HG02610       HG02642 
-5.252571e-01 -7.117559e-01  3.780461e-01 -5.318359e-02  5.427009e-02 
      HG02722       HG02810       HG02851       HG02855       HG02890 
-2.105005e-01  2.624804e-02  2.383038e-01 -8.747320e-02  9.742056e-01 
      HG03046       HG02562       HG02594       HG02611       HG02643 
 1.052536e-02  1.030876e-01 -3.645886e-02  1.988345e-01  8.699922e-02 
      HG02757       HG02811       HG02852       HG02860       HG02891 
 8.872759e-01 -5.987725e-01  5.043034e-01 -8.762588e-01 -4.791361e-02 
      HG03048       HG02568       HG02595       HG02613       HG02645 
 7.459280e-01  9.682564e-01 -2.568254e-01 -2.974965e-01  3.350721e-01 
      HG02759       HG02813       HG02854       HG02861       HG02895 
-5.575027e-02  9.382367e-02 -8.522719e-01 -2.567643e-01 -8.541655e-01 
      HG03049       HG02570       HG02614       HG02646       HG02760 
-6.820142e-01  4.224773e-01  6.859987e-01 -2.287753e-01 -5.822670e-02 
      HG02814       HG02870       HG02896       HG02571       HG02620 
 5.967614e-01  7.676715e-01  2.185069e-02 -2.783416e-03 -9.217840e-01 
      HG02666       HG02676       HG02763       HG02816       HG02878 
-6.212299e-01 -3.825453e-01  6.634032e-01  1.205261e-01  1.678806e-01 
      HG02982       HG03241       HG02573       HG02621       HG02667 
-9.402395e-02 -1.161950e+00 -3.639884e-01  5.492802e-01  5.101067e-01 
      HG02678       HG02768       HG03240       HG02817       HG02879 
-1.953209e-01  3.429433e-01  7.097931e-02  1.131365e+00  8.475543e-01 
      HG02983       HG03246 
-1.465219e-01  2.932822e-01

In [32]:
install.packages("ggplot2")
library(ggplot2)

pop="ESN"
r_file = sprintf('%s/eoutliers/%s_exprResiduals.tsv', data_dir, pop)

# Write out the residuals
resids = read.table(r_file, header=T, row.names=1)
resids = data.frame(t(resids))

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [35]:
# ggplot(resids, aes(x=ENSG00000227232.5)) +  geom_histogram()
# ggplot(resids, aes(x=ENSG00000279457.4)) +  geom_histogram()

In [38]:
model$residuals

HG02941     HG02976     HG03114     HG03135     HG03189     HG03271 
 1.28620986  0.09517781 -0.56483295 -0.98152323 -0.86660353  0.27340020 
    HG03313     HG03367     HG02943     HG02977     HG03115     HG03136 
 1.15570805  1.84581228 -0.60016152  0.36022590  1.24972308  0.57575629 
    HG03190     HG03279     HG03280     HG03342     HG03369     HG02981 
 0.79866625 -0.66281871  1.45999878 -1.38466743 -0.81170084  0.56394649 
    HG03117     HG03139     HG03291     HG03343     HG03370     HG02946 
-1.00044060  0.66985292 -0.87781413  0.16201296  1.03768584 -0.77909504 
    HG03118     HG03157     HG03195     HG03294     HG03351     HG03372 
 1.00567383  0.73618760 -1.06354636 -0.22085854 -0.47299408 -0.78916378 
    HG02947     HG03099     HG03120     HG03159     HG03196     HG03295 
 0.65458988 -0.77265545  0.55041155 -1.22616954  1.07295356  1.07713672 
    HG03352     HG03499     HG02952     HG03100     HG03121     HG03124 
 0.05362686 -0.93626991 -1.20449322  0.95033290 -1.24160174  0.52823887 
    HG03160     HG03198     HG03297     HG03354     HG02953     HG03103 
 0.70473047 -0.88713051 -0.27888672 -0.11081244  0.73880210  0.69847599 
    HG03123     HG03126     HG03162     HG03199     HG03514     HG02968 
-1.16993239 -1.17596771 -1.57891712  0.79739466 -0.76423428  0.39662295 
    HG03105     HG03127     HG03163     HG03202     HG03300     HG03515 
-0.85623342  1.22084605  0.10009353  0.67680690 -1.04745196  0.56155414 
    HG02970     HG03108     HG03129     HG03166     HG03265     HG03301 
-0.79832001 -1.22107957 -0.91202453  0.10455327  0.48745561  0.93865427 
    HG03517     HG02922     HG02971     HG03109     HG03130     HG03168 
-1.78923990 -0.99170721  0.68419375  1.57894213  0.91873240 -1.42367574 
    HG03169     HG03267     HG03303     HG03518     HG02923     HG02973 
 0.41521506 -0.84946944 -0.86530409  0.95282555  0.81888591 -0.66999162 
    HG03111     HG03172     HG03268     HG03304     HG03363     HG02938 
-0.54823234  0.78963466  0.21466402  0.82897612 -1.29769825  1.07845967 
    HG02974     HG03112     HG03133     HG02944     HG03132     HG03193 
-0.62274959  0.78715632  0.62820675  0.79319312 -1.17635441  0.71797129 
    HG03298     HG03520     HG03175     HG03270     HG03311     HG03366 
 0.77771425 -1.27982232  0.78505669 -0.97097687  0.67840878 -0.32284025 
    HG03521 
 1.02890838

### Label outliers

ERROR: Error in quantile.default(resids): missing values and NaN's not allowed if 'na.rm' is FALSE


In [3]:
QTHRESH = F

if (QTHRESH) {
    q = quantile(resids)
    z_thresh = as.numeric(abs(q["75%"] - q["25%"]))
} else {
    z_thresh = 3
}

# AFGR individuals have more on average, so we can choose a higher threshold than Watershed
z_ind_filt = 75
r_file = sprintf('%s/eoutliers/%s_exprResiduals.tsv', data_dir, pop)

# Write out the residuals
resids = read.table(r_file, header=T, row.names=1)
resids = data.frame(t(resids))

w = abs(resids) > z_thresh
w = data.frame(sapply(data.frame(w), function(x) as.integer(x)))
rownames(w) = rownames(resids)
indiv_noutliers = data.frame(rowSums(w))
indiv_keep = rownames(indiv_noutliers)[which(indiv_noutliers < z_ind_filt)]

mean(indiv_noutliers[,])
sqrt(var(indiv_noutliers[,]))
median(indiv_noutliers[,])
print(sprintf("Keeping %i / %i individuals with z_thresh = %#.2f, max outliers = %i",length(indiv_keep), length(rownames(w)), z_thresh, z_ind_filt))
write.table(indiv_keep, sprintf('%s/eoutliers/ids_outlier_filtered_%s_t%#.2ff%i.txt', data_dir, pop, z_thresh, z_ind_filt), quote = F, sep = '\t', col.names = F, row.names = F)
write.table(w, sprintf('%s/eoutliers/eOutlier_scores_%s_t%#.2f.txt', data_dir, pop, z_thresh))



[1] 77.95876

[1] 94.7291

[1] 53

[1] "Keeping 63 / 97 individuals with z_thresh = 3.00, max outliers = 75"


In [164]:
# Create a combined file for AFR
pops = c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")
afr_id_filt = data.frame()
afr_eout_scores = data.frame()
for (pop in pops) {
    afr_id_filt = rbind(afr_id_filt,
          read.table(sprintf('%s/ids_outlier_filtered_%s_t%if%i.txt', data_dir, pop, z_thresh, z_ind_filt), sep=" "))
    afr_eout_scores = rbind(afr_eout_scores,
          read.table(sprintf('%s/eOutlier_scores_%s_t%i.txt', data_dir, pop, z_thresh), sep=" "))
}
write.table(afr_id_filt, sprintf('%s/ids_outlier_filtered_AFR_t%if%i.txt', data_dir, z_thresh, z_ind_filt), quote = F, sep = '\t', col.names = F, row.names = F)
write.table(afr_eout_scores, sprintf('%s/eOutlier_scores_AFR_t%i.txt', data_dir, z_thresh))
